In [1]:
import torch
import tqdm

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [ ]:
from datasets import Dataset
import pandas as pd
from transformers import Pipeline
from transformers.pipelines.pt_utils import KeyDataset

from transformers import (
    AutoConfig,
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
)

/home/jupyter/.local/lib/python3.10/site-packages/transformers/utils/hub.py:105: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
2025-05-30 21:26:20.035605: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-30 21:26:20.087724: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-05-30 21:26:21.333817: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
# test = '/home/jupyter/datasphere/project/rugec/data/RULEC-GEC.test.tsv'
test = '/home/jupyter/datasphere/project/rugec/data/RULEC-GEC.dev.tsv'
rulec_test = pd.read_csv(test, delimiter='\t')

In [ ]:
test_ds = Dataset.from_dict({'corrupt_sent':rulec_test['corrupt_sent'],'correct_sent' : rulec_test['correct_sent']})

In [5]:
for i in test_ds.take(1):
    print(i)

{'corrupt_sent': 'Экскурсия прошла великолепно , Владимир Анатольевич просто изумительный экскурсовод .', 'correct_sent': 'Экскурсия прошла великолепно , Владимир Анатольевич просто изумительный экскурсовод .'}


In [ ]:
# name = 'mika5883/pretrain_rugec_msu'
# model_name = 'mika5883/gec_t5_dpo'
# model_name = 'mika5883/gec_t5_dpo_A_v1'
# name = 'mika5883/finetune_rugec_msu'
# model_name = 'mika5883/ft_rugec_A'
# model_name = 'mika5883/Full_Train_Dev'
# model_name = 'mika5883/gec_t5_dpo_A_v2'
# model_name = 'mika5883/RULEC_Tr_Dev_NVP5000'
model_name = 'mika5883/finetune_rugec_Ae' 
config = AutoConfig.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers


In [ ]:
# can try sampling diverse outputs for some purposes but it doesn't give the best results on its own.


# def enable_dropout(model):
#     """Function to enable the dropout layers during test-time"""
#     for module in model.modules():
#         if isinstance(module, torch.nn.Dropout):
#             module.train()
# model.eval()          
# enable_dropout(model) 

In [ ]:
def tokenize_function(examples, prefix='grammar: '):
    # inputs = [f'grammar: {each}' for each in examples['corrupt_sent']]
    inputs = [f'{prefix}{each}' for each in examples]
    model_inputs = tokenizer(inputs, max_length=128, padding='max_length', truncation=True, return_tensors='pt')
    model_inputs['inputs'] = inputs
    return model_inputs

In [ ]:
class MyPipeline(Pipeline):
    def _sanitize_parameters(self, **kwargs):
        preprocess_kwargs = {}
        if "maybe_arg" in kwargs:
            preprocess_kwargs["maybe_arg"] = kwargs["maybe_arg"]
        return preprocess_kwargs, {}, {}

    def preprocess(self, inputs, maybe_arg=2):
        if isinstance(inputs, str):
            inputs = [inputs]
        model_inputs = tokenize_function(inputs).input_ids.to(self.device)
        return {"inputs": model_inputs}

    def _forward(self, model_inputs):
        # with torch.no_grad():
        outputs = self.model.generate(
            **model_inputs, 
            max_new_tokens=128, 
            num_return_sequences=6, 
            num_beams=6,
            early_stopping=True
        )
        return outputs

    def postprocess(self, model_outputs):
        return tokenizer.batch_decode(
            model_outputs, 
            skip_special_tokens=True, 
            clean_up_tokenization_spaces=False
        )

    def __call__(self, inputs):
        return self.postprocess(self._forward(self.preprocess(inputs)))


In [11]:
gram = MyPipeline(model=model, tokenizer=tokenizer, device=0)

Device set to use cuda:0


In [14]:
pr = gram.preprocess('grammar: Мнагие люди ашибочно пологают , что земле 2000 лет .')
outs = gram.forward(pr)
gram.postprocess(outs)

[': Многие люди ошибочно полагают , что земле 2000 лет .',
 'Многие люди ошибочно полагают , что земле 2000 лет .',
 'Многие люди ошибочно полагают , что земле 2000 лет .',
 'Мнагие люди ошибочно полагают , что земле 2000 лет .',
 'Многим людям ошибочно полагают , что земле 2000 лет .',
 'Некоторые люди ошибочно полагают , что земле 2000 лет .']

In [ ]:
from math import ceil

corrected_sents = []
batch_size = 64
total_count = len(test_ds)
chunks = ceil(total_count / batch_size)

def div(outs, n=6):
    #grouping corrections per sentence since len(outputs) == batch_size * num_return_sequences
    return [outs[i:i+n] for i in range(0, len(outs), n)]

for i in tqdm.tqdm(range(chunks), desc="Correcting"):
    batch = KeyDataset(test_ds, 'corrupt_sent')[i*batch_size:(i+1)*batch_size]
    outputs = gram(batch)
    corrected_sents.append(div(outputs))

torch.cuda.empty_cache()

Correcting: 100%|██████████| 40/40 [04:21<00:00,  6.54s/it]


In [15]:
cors = [b for a in corrected_sents for b in a]
len(cors)

2500

In [16]:
corrupt_corrected_pairs = [each for each in zip(KeyDataset(test_ds, 'corrupt_sent'), cors)]
# corrupt_corrected_pairs = [[each[0], each[1]] for each in corrupt_corrected_pairs]
corrupt_corrected_pairs[0]

('Экскурсия прошла великолепно , Владимир Анатольевич просто изумительный экскурсовод .',
 ['Экскурсия прошла великолепно , Владимир Анатольевич просто изумительный экскурсовод .',
  'Экскурсия прошла великолепно , Владимир Анатольевич просто великолепный экскурсовод .',
  'Экскурсия прошла великолепно , Владимир Анатольевич просто удивительный экскурсовод .',
  'Экскурсия прошла прекрасно , Владимир Анатольевич просто изумительный экскурсовод .',
  'Экскурсия прошла великолепно , Владимир Анатольевич просто изумительный гидовод .',
  'Экскурсия прошла великолепно , Владимир Анатольевич просто замечательный экскурсовод .'])

In [19]:
len(corrected_sents[0])

64

In [17]:
rulec_test.iloc[0]['correct_sent']

'Экскурсия прошла великолепно , Владимир Анатольевич просто изумительный экскурсовод .'

In [ ]:
df = pd.DataFrame(corrupt_corrected_pairs, columns=['corrupt', 'hypotheses'])
# df.hypotheses = df.hypotheses.map(lambda x: [x]) # in case there's only one output
df.head()

,corrupt,hypotheses
0,"Экскурсия прошла великолепно , Владимир Анатол...","[Экскурсия прошла великолепно , Владимир Анато..."
1,"Курсовая может быть о любой теме , которую обс...","[Курсовая может быть о любой теме , которую об..."
2,"Я вижу , я слышу всё вокруг меня .","[Я вижу , я слышу всё вокруг меня ., Я вижу , ..."
3,Служить в Российской армии ( для мужчин ),"[Служить в Российской армии ( для мужчин ), Сл..."
4,Вряд ли все мир обеспокоился бы из-за бунта в ...,[Вряд ли весь мир обеспокоился бы из-за бунта ...


In [19]:
df.to_csv('dev_results/finetune_rugec_Ae_mult.tsv', sep='\t', index=False)